In [3]:
import pandas as pd
import numpy as np
import os
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

def create_final_inference_file(data_path, output_file):
    """
    一个完整的流程：加载数据、拟合处理器、按最终规则采样、处理数据，并保存为.pt文件。
    """
    # --- 步骤 1: 加载数据并进行清洗和标签映射 ---
    print("步骤 1/5: 加载数据并进行清洗和标签映射...")
    parquet_files = [os.path.join(data_path, f) for f in os.listdir(data_path)
                     if f.endswith('.parquet')]
    if not parquet_files:
        raise FileNotFoundError(f"错误：在路径 '{data_path}' 下没有找到任何 .parquet 文件。")
    
    df = pd.concat([pd.read_parquet(f) for f in tqdm(parquet_files, desc="加载文件")], ignore_index=True)
    
    # 清理列名中的空格
    df.rename(columns={col: col.strip() for col in df.columns}, inplace=True)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    
    # 清理原始标签中的异常字符
    df['Label'] = df['Label'].astype(str).str.replace(r'[^a-zA-Z0-9\s-]', '', regex=True).str.strip()

    # 映射到7个主要类别
    multi_class_mapping = {
        'DoS Hulk': 'DoS', 'DoS GoldenEye': 'DoS', 'DoS slowloris': 'DoS',
        'DoS Slowhttptest': 'DoS', 'FTP-Patator': 'Brute_Force',
        'SSH-Patator': 'Brute_Force', 'Web Attack Brute Force': 'Web_Attack',
        'Web Attack XSS': 'Web_Attack', 'Web Attack Sql Injection': 'Web_Attack',
        'PortScan': 'PortScan', 'Bot': 'Bot', 'Infiltration': 'Rare_Attacks',
        'Heartbleed': 'Rare_Attacks'
    }
    df['Multi_Label'] = df['Label'].replace(multi_class_mapping)

    # 关键步骤：删除模型未学习的小众类别
    original_rows = len(df)
    df = df[~df['Multi_Label'].isin(['Rare_Attacks'])]
    print(f"移除了 {original_rows - len(df)} 条 'Rare_Attacks' (Infiltration, Heartbleed) 数据。")
    
    df.drop_duplicates(inplace=True)
    print("数据加载和预处理完成。")

    # --- 步骤 2: 准备并拟合数据处理器 ---
    print("\n步骤 2/5: 准备并拟合数据处理器以匹配模型...")
    feature_columns = [col for col in df.columns if col not in ['Label', 'Multi_Label']]
    X = df[feature_columns]
    y = df['Multi_Label']
    
    label_encoder = LabelEncoder()
    label_encoder.fit(y)
    print(f"标签编码器拟合完成，最终类别: {list(label_encoder.classes_)}")

    scaler = StandardScaler()
    scaler.fit(X)
    print("标准化处理器(Scaler)拟合完成。")
    
    # --- 步骤 3: 从处理后的数据中按最终规则采样 ---
    print("\n步骤 3/5: 按最终规则进行分层采样...")
    def get_sample_size(group_name):
        return 130 if group_name == 'Benign' else 10

    sampled_dfs = []
    for group_name, group_df in df.groupby('Multi_Label'):
        n = get_sample_size(group_name)
        sampled_dfs.append(group_df.sample(n=n, random_state=42))
        print(f"  - 类别 '{group_name}': 成功抽取 {n} 条数据")

    sampled_df = pd.concat(sampled_dfs, ignore_index=True)
    print("采样完成。")

    # --- 步骤 4: 处理采样数据并转换为Tensor ---
    print("\n步骤 4/5: 处理采样数据并转换为PyTorch Tensor...")
    x_sample_raw = sampled_df[feature_columns]
    y_sample_str = sampled_df['Multi_Label']

    x_sample_scaled = scaler.transform(x_sample_raw)
    y_sample_encoded = label_encoder.transform(y_sample_str)
    
    features_tensor = torch.tensor(x_sample_scaled, dtype=torch.float32)
    labels_tensor = torch.tensor(y_sample_encoded, dtype=torch.long)
    print(f"数据已转换为Tensor。特征形状: {features_tensor.shape}, 标签形状: {labels_tensor.shape}")

    # --- 步骤 5: 保存为.pt文件 ---
    print(f"\n步骤 5/5: 保存到文件 '{output_file}'...")
    data_to_save = {
        'features': features_tensor,
        'labels': labels_tensor,
        'class_names': list(label_encoder.classes_)
    }
    torch.save(data_to_save, output_file)
    print("保存成功！")

# --- 主程序入口 ---
if __name__ == "__main__":
    DATA_DIRECTORY = '/kaggle/input/cicids2017' 
    OUTPUT_FILE = 'inference_data.pt'

    try:
        create_final_inference_file(DATA_DIRECTORY, OUTPUT_FILE)
        print(f"\n🎉 最终文件 '{OUTPUT_FILE}' 已成功生成！")
        
    except Exception as e:
        print(f"\n❌ 发生错误: {e}")

步骤 1/5: 加载数据并进行清洗和标签映射...


加载文件: 100%|██████████| 8/8 [00:00<00:00,  8.72it/s]


移除了 47 条 'Rare_Attacks' (Infiltration, Heartbleed) 数据。
数据加载和预处理完成。

步骤 2/5: 准备并拟合数据处理器以匹配模型...
标签编码器拟合完成，最终类别: ['Benign', 'Bot', 'Brute_Force', 'DDoS', 'DoS', 'PortScan', 'Web Attack  Brute Force', 'Web Attack  Sql Injection', 'Web Attack  XSS']
标准化处理器(Scaler)拟合完成。

步骤 3/5: 按最终规则进行分层采样...
  - 类别 'Benign': 成功抽取 130 条数据
  - 类别 'Bot': 成功抽取 10 条数据
  - 类别 'Brute_Force': 成功抽取 10 条数据
  - 类别 'DDoS': 成功抽取 10 条数据
  - 类别 'DoS': 成功抽取 10 条数据
  - 类别 'PortScan': 成功抽取 10 条数据
  - 类别 'Web Attack  Brute Force': 成功抽取 10 条数据
  - 类别 'Web Attack  Sql Injection': 成功抽取 10 条数据
  - 类别 'Web Attack  XSS': 成功抽取 10 条数据
采样完成。

步骤 4/5: 处理采样数据并转换为PyTorch Tensor...
数据已转换为Tensor。特征形状: torch.Size([210, 77]), 标签形状: torch.Size([210])

步骤 5/5: 保存到文件 'inference_data.pt'...
保存成功！

🎉 最终文件 'inference_data.pt' 已成功生成！
